# Offline Feature Extraction using ERD / ERS

- ## ERS - Event Related Synchonization: 
$$ERS[\%] = \big( P_{ERS} - P_{rest} \big) / P_{rest} * 100\% $$


- ## ERD - Event Related Desynchonization:
$$ERD[\%] = \big( P_{ERD} - P_{rest} \big) / P_{rest} * 100\% $$

In [32]:
import numpy as np
import scipy.io
import scipy.integrate
import pylab
import matplotlib.pyplot as plt 
import mne
from mne.time_frequency import psd_multitaper
import time
 
%matplotlib inline

mne.set_log_level('WARNING')

## Set Parameters

In [33]:
freq_s = 250    # [Hz] Sampling Frequency

trial_len = 4    # [sec] length of a trial 
rest_len = 1    # [sec] time before trial start
erd_start = 1    # [sec] after trial time start
erd_len = 4    # [sec] length of erd window
ers_start = trial_len    # [sec] after trial time start
ers_len = rest_len    # [sec] length of ers window

a_band = [8, 12]    # [Hz] alpha band
b_band = [13, 26]    # # [Hz] beta band

## Read Raw Data

In [34]:
# Load Raw EEG
raw_fname = '/Users/leonardrychly/Dropbox/[TUM]/4. WiSe 1617/Masterarbeit/Code/eeg_feature_extraction/exp_0-raw.fif'
raw = mne.io.read_raw_fif(raw_fname, add_eeg_ref=False).load_data()

# Remove EOG channels (channels 4-6)
raw.drop_channels(['eog_4','eog_5', 'eog_6'])

<Raw  |  exp_0-raw.fif, n_channels x n_times : 3 x 604803 (2419.2 sec), ~13.9 MB, data loaded>

#### Remove Artifacts

In [35]:
# Load data containing artifacts
data_path = "/Users/leonardrychly/Dropbox/[TUM]/4. WiSe 1617/Masterarbeit/Code/eeg_feature_extraction/B01T.mat"
mat = scipy.io.loadmat(data_path)['data']

# Get artifacts
exp_arr = mat[0,0]
# explanation:   exp_arr[0][0][data]
exp_dict = {'X': exp_arr[0][0][0].T,
            'trial': exp_arr[0][0][1],
            'y': exp_arr[0][0][2],
            'fs': exp_arr[0][0][3].flatten(),
             'classes': exp_arr[0][0][4].flatten(),
             'artifacts': exp_arr[0][0][5],
             'gender': exp_arr[0][0][6],
             'age': exp_arr[0][0][7].flatten()
             }

# Remove from 'trials' and 'y'
artifact_idx = exp_dict['artifacts'].nonzero()[0]
exp_dict['trial'] = np.delete(exp_dict['trial'], artifact_idx)
exp_dict['y'] = np.delete(exp_dict['y'], artifact_idx)

# Functions

# Create Feature Matrix

#### X.shape = (n_samples, n_subbands*n_channels)

In [36]:
# Init X
n_trials   = len(exp_dict['trial'])
n_erp = 2
n_channels = len(raw.ch_names)
X = np.zeros((n_trials, n_erp * n_channels))
# Init y
y = np.zeros(len(exp_dict['y']))

print("X.shape: ", X.shape)
print("y.shape: ", y.shape)

X.shape:  (102, 6)
y.shape:  (102,)


In [37]:
filter_len = int((trial_len + rest_len) * freq_s)

t0 = time.time()

for t, trial_start in enumerate(exp_dict['trial']):    # t: iterate over trials

    ### CROP ##########################################################################
    # trial idx to time point
    trial_stop = trial_start + (trial_len * freq_s)
    trial_start_t = trial_start / freq_s
    trial_stop_t = (trial_start / freq_s + trial_len) 
    
    # Crop Rest window
    tmin_rest = trial_start_t - rest_len
    tmax_rest = trial_start_t
    raw_temp = raw.copy()
    raw_rest = raw_temp.crop(tmin=tmin_rest, tmax=tmax_rest)
    # Crop ERD (alpha) window
    tmin_erd = trial_start_t + erd_start
    tmax_erd = trial_start_t + erd_start + erd_len
    raw_temp = raw.copy()
    raw_erd = raw_temp.crop(tmin=tmin_erd, tmax=tmax_erd)
    # Crop ERS (beta) window
    tmin_ers = trial_start_t + ers_start
    tmax_ers = trial_start_t + ers_start + ers_len
    raw_temp = raw.copy()
    raw_ers = raw_temp.crop(tmin=tmin_ers, tmax=tmax_ers)
    
    
    ### FILTER croped windows ##########################################################
    
    # Filter Rest
    filter_len_rest = int((tmax_rest - tmin_rest) * freq_s)
    # alpha
    raw_temp = raw_rest.copy()
    raw_rest_a = raw_temp.filter(l_freq=a_band[0], h_freq=a_band[1], 
                                 filter_length=filter_len_rest, l_trans_bandwidth='auto',
                                 h_trans_bandwidth='auto', n_jobs=1, method='fir', 
                                 iir_params=None, phase='zero', fir_window='hamming')
    # beta
    raw_temp = raw_rest.copy()
    raw_rest_b = raw_temp.filter(l_freq=b_band[0], h_freq=b_band[1], 
                                 filter_length=filter_len_rest, l_trans_bandwidth='auto',
                                 h_trans_bandwidth='auto', n_jobs=1, method='fir', 
                                 iir_params=None, phase='zero', fir_window='hamming')

    # Filter ERD
    filter_len_erd = int((tmax_erd - tmin_erd) * freq_s)
    raw_temp = raw_erd.copy()
    raw_erd_a = raw_temp.filter(l_freq=a_band[0], h_freq=a_band[1], 
                                filter_length=filter_len_erd, l_trans_bandwidth='auto',
                                h_trans_bandwidth='auto', n_jobs=1, method='fir', 
                                iir_params=None, phase='zero', fir_window='hamming')

    # Filter ERS
    filter_len_ers = int((tmax_ers - tmin_ers) * freq_s)

    raw_temp = raw_ers.copy()
    raw_ers_b = raw_temp.filter(l_freq=b_band[0], h_freq=b_band[1], 
                                filter_length=filter_len_rest, l_trans_bandwidth='auto',
                                h_trans_bandwidth='auto', n_jobs=1, method='fir', 
                                iir_params=None, phase='zero', fir_window='hamming')
    
    
    ### Power ############################################################################
    
    # Paramters
    f_min_rest_a, f_max_rest_a = 0,  20
    f_min_rest_a, f_max_rest_a = 10, 30
    f_min_erd,    f_max_erd    = 0,  20
    f_min_ers,    f_max_ers    = 10, 30



    
    # PSD for Rest (alpha)
    raw_temp = raw_rest_a.copy()
    psds_rest_a, freqs_rest_a = psd_multitaper(raw_temp, low_bias=True, 
                                               tmin=0, tmax=rest_len, 
                                               fmin=f_min_erd, fmax=f_max_erd, proj=True)

    # PSD for Rest (beta)
    raw_temp = raw_rest_b.copy()
    psds_rest_b, freqs_rest_b = psd_multitaper(raw_temp, low_bias=True, 
                                               tmin=0, tmax=rest_len, 
                                               fmin=f_min_erd, fmax=f_max_erd, proj=True)
    
    # PSD for ERD
    raw_temp = raw_erd_a.copy()
    psds_erd, freqs_erd = psd_multitaper(raw_temp, low_bias=True, 
                                         tmin=0, tmax=erd_len, 
                                         fmin=f_min_erd, fmax=f_max_erd, proj=True)
    
    # PSD for ERS
    raw_temp = raw_ers_b.copy()
    psds_ers, freqs_ers = psd_multitaper(raw_temp, low_bias=True, 
                                         tmin=0, tmax=ers_len, 
                                         fmin=f_min_ers, fmax=f_max_ers, proj=True)

    # Power for Rest (alpha)
    powers_rest_a = np.zeros(len(psds_rest_a))
    for i, psd in enumerate(psds_rest_a):
        powers_rest_a[i] = scipy.integrate.simps(psd)

    # Power for Rest (beta)
    powers_rest_b = np.zeros(len(psds_rest_b))
    for i, psd in enumerate(psds_rest_b):
        powers_rest_b[i] = scipy.integrate.simps(psd)
        
    # Power for each channel in ERD 
    powers_erd = np.zeros(len(psds_erd))
    for i, psd in enumerate(psds_erd):
        powers_erd[i] = scipy.integrate.simps(psd)
        
    # Power for each channel in ERS
    powers_ers = np.zeros(len(psds_ers))
    for i, psd in enumerate(psds_ers):
        powers_ers[i] = scipy.integrate.simps(psd)

    # ERD
    erd = (powers_erd - powers_rest_a) / powers_rest_a * 100
    
    # ERS
    ers = (powers_ers - powers_rest_b) / powers_rest_b * 100

            
    ### Add to Feature Matrix #############################################################
        
    # init row
    erd_len = len(erd) + len(ers)
    row = np.zeros(erd_len)

    # add erps new row
    for c in range(len(erd)):    # c: iterate over channels
        row[c] = erd[c]
        row[c + len(ers)] = ers[c]
        
    # add row to feat mat
    X[t, :] = row
            
    # add label to y
    y[t] = exp_dict['y'][t]
               
    
print("Creating Feature Matrix takes {:0.2f} seconds".format(time.time()-t0))
print("X.shape: ", X.shape)
print("y.shape: ", y.shape)

Attenuation at stop frequency 6.0Hz is only 17.0dB. Increase filter_length for higher attenuation.


<ipython-input-37-e77a7ff5d348>:39: RuntimeWarning: Attenuation at stop frequency 6.0Hz is only 17.0dB. Increase filter_length for higher attenuation.
  iir_params=None, phase='zero', fir_window='hamming')


Attenuation at stop frequency 6.0Hz is only 17.0dB. Increase filter_length for higher attenuation.


<ipython-input-37-e77a7ff5d348>:39: RuntimeWarning: Attenuation at stop frequency 6.0Hz is only 17.0dB. Increase filter_length for higher attenuation.
  iir_params=None, phase='zero', fir_window='hamming')


Attenuation at stop frequency 6.0Hz is only 17.0dB. Increase filter_length for higher attenuation.


...

Attenuation at stop frequency 6.0Hz is only 17.0dB. Increase filter_length for higher attenuation.


<ipython-input-37-e77a7ff5d348>:39: RuntimeWarning: Attenuation at stop frequency 6.0Hz is only 17.0dB. Increase filter_length for higher attenuation.
  iir_params=None, phase='zero', fir_window='hamming')


Attenuation at stop frequency 6.0Hz is only 17.0dB. Increase filter_length for higher attenuation.


<ipython-input-37-e77a7ff5d348>:39: RuntimeWarning: Attenuation at stop frequency 6.0Hz is only 17.0dB. Increase filter_length for higher attenuation.
  iir_params=None, phase='zero', fir_window='hamming')


Creating Feature Matrix takes 139.28 seconds
X.shape:  (102, 6)
y.shape:  (102,)


In [30]:
np.save('X_feat_mat_erp.npy', X)

In [31]:
np.save('y_label_vect_erp.npy', y)